# imports 

`-` 충분통계량을 따지는 방법

1. 정의에 넣고 따짐
2. pdf를 보고 따짐 

`-` 충분통계량의 1:1 함수는 충분통계량이다. 

`-` MLE가 유일하게 존재하면 MLE는 충분통계량의 함수이다. 

`-` 라오-블랙웰정리 [@ 4.5.] 충분통계량 $Y$를 이용하면 $\hat{\theta}$를 개선할 수 있다. 구체적으로는 $\mathbb{E}(\hat{\theta}|Y)$를 이용하면 된다. 만약에 충분통계량 $Y$가 완비충분통계량이라면 $\mathbb{E}(\hat{\theta}|Y)$는 UMVUE 가 된다. (레만쉐페의 정리 @ 4.6)

`-` 레마: $Y$가 CSS이고 $\hat{\theta}$이 $Y$를 이용한 unbiased estimator라면 ${\theta}$이 UMVUE 이다. p221

`-` Exp에서는 CSS를 쉽게 구할 수 있다. 

`-` 유일한 MLE가 존재하고 그것이 충분통계량이라면, 그것은 MSS이다. 

`-` 따라서 지수족에서는 MSS = CSS 이다. 

In [3]:
50/3

16.666666666666668

# 예제1

## 문제소개

In [111]:
D = {'doc1':['손흥민','골'],'doc2':['골','확률'],'doc3':['확률','데이터과학']}
D

{'doc1': ['손흥민', '골'], 'doc2': ['골', '확률'], 'doc3': ['확률', '데이터과학']}

- 여기에서 $D$는 문서들의 집합, 즉 코퍼스를 의미한다. 

`-` 토픽

- 토픽1 = 축구 : '골', '손흥민' 와 같은 단어가 쓰임. 
- 토픽2 = 통계 : '데이터과학', '확률' 와 같은 단어가 쓰임. 

`-` 문서
    
- 문서1은 축구를 주제로 쓰여진 글인듯 하다.
- 문서3은 데이터과학을 주제로 쓰여진 글인듯 하다. 
- 문서2는 축구와 데이터과학이 혼합된 주제인듯 하다. 

`-` 원하는 것 (1): 문서별로 토픽이 얼마나 들어있는지 알고싶다. 

In [108]:
θ = {'doc1':[1,1],'doc2':[1,2],'doc3':[2,2]}
θ

{'doc1': [1, 1], 'doc2': [1, 2], 'doc3': [2, 2]}

`-` $D$를 입력으로 넣으면 $\theta$ 가 나오면 좋겠음 

`-` 비판

- 그런데 생각해보니까 "손흥민"이라는 단어는 완전 축구단어
- "골"이라는 단어는 가끔 데이터과학과 섞일 수 있는 단어 
- "손흥민"이라는 단어는 축구토픽에서 나올 확률이 90퍼 이상, "골"은 축구토픽에서 나올 확률이 80퍼정도일수도 있잖음? 

`-` 특정한 하나의 ${\boldsymbol \theta}$ 값이 아니라 ${\boldsymbol \theta}$의 분포 ${\boldsymbol \pi}_{\theta}$ 즉 

- ${\boldsymbol \theta} \sim {\boldsymbol \pi}_{\theta}$

와 같은 구조로 ${\boldsymbol \theta}$를 뽑아낼 수 있는 분포를 알아내는게 더 좋지 않을까? 

`-` 낙관론자: MCMC로 ${\boldsymbol \theta}$를 막 뽑아서 그것들을 가지고 추론하면 어떨까? 

`-` 비관론자: 아니 ${\boldsymbol \pi}_{\boldsymbol \theta}$를 모르잖아? 그런데 어떻게 뽑아?

## 가능한 해결책

`-` 이론적인 근거는 없는데요 (있지만 현재는 논의하지 않겠음) 되게 그럴듯한 해결책이 있습니다. 

`-` 문제의 단순화:
    
- 결국 길이가 6인 랜덤벡터 ${\boldsymbol \theta}=(\theta[0],\dots,\theta[5])$ 를 뽑는 일이다. 

### ***단계1: 아무값이나 넣어서 ${\boldsymbol \theta}_0$를 초기화한다.***

`-` 아무값이나 셋팅 

In [128]:
θ = {'doc1':[2,1],'doc2':[1,2],'doc3':[2,1]}
θ

{'doc1': [2, 1], 'doc2': [1, 2], 'doc3': [2, 1]}

- ${\boldsymbol \theta}_0 = [2,1,1,2,2,1]$ 이라고 생각하자. 

`-` 임의의 초기값이 셋팅된 상황은 아래와 같다. 

In [129]:
D

{'doc1': ['손흥민', '골'], 'doc2': ['골', '확률'], 'doc3': ['확률', '데이터과학']}

***1.문서당 토픽비율조사***

- doc1:[손흥민,골]=[2,1] 
- doc2:[골,확률]=[1,2]
- doc3:[확률,데이터과학]=[2,1]

***2.토픽별로 자주등장하는 단어조사***

- 토픽1: 골,골,데이터과학 --> 축구?? (데이터과학이 살짝 애매하긴 하네?)
- 토픽2: 손흥민,확률,확률 --> 통계?? (손흥민이 살짝 애매하긴 하네?)

### ***단계2: ${\boldsymbol \theta}_0[0], {\boldsymbol \theta}_0[1] \cdots, {\boldsymbol \theta}_0[5]$ 을 순서대로 샘플링***

***예비개념: MCMC에서 베타분포를 뽑았던 그때로 돌아가보자*** 

1. 임의의 초기값 $x$ 생성 
2. 새로운 값으로 $y$를 고려 (추천받을 수도 있고 그냥 고려할수도 있음)
3. $x$가 그럴듯한지, $y$가 그럴듯한지 판단하고 $x'$의 값은 어떠한확률로 $x,y$ 중에서 선택

#### stage0: ***${\boldsymbol \theta}_0[0]$의 고민: 나는 토픽1인가 토픽2인가?***

In [133]:
θ

{'doc1': [2, 1], 'doc2': [1, 2], 'doc3': [2, 1]}

In [134]:
D

{'doc1': ['손흥민', '골'], 'doc2': ['골', '확률'], 'doc3': ['확률', '데이터과학']}

In [135]:
θ['doc1'][0], D['doc1'][0]

(2, '손흥민')

`-` 지금 우리의 관심은 `doc1-손흥민`에 해당하는 토픽으로 `토픽=1`이라 생각하는 것이 그럴듯한지, `토픽=2`라고 생각하는게 그럴듯한지 다시 따져보고 싶다. 

`-` 가정: `doc1-손흥민`에 대응하는 토픽분류를 제외하고 나머지는 모두 올바르게 토픽이 분류되었다고 가정하자. 

`문서눈치`: 나($=\theta_0$)는 `doc1`에 있음. `doc1`에는 다른단어들도 있음. 그렇다면 `doc1`에 있는 다른 단어들의 토픽들은 어떤지 살펴보자.

In [119]:
θ['doc1']

[2, 1]

- `doc1`에는 현재 관심이 있는 `doc1-손흥민`을 제외하고 다른단어는 하나뿐인데 그 단어는 토픽1로 분류됨 $\to$ 나도 토픽1인가? 

`단어눈치`: 나($=\theta_0$)는 `골`이라는 단어임. 토픽1에 있는 다른단어를 찾아볼까? 

- 토픽1:[골,골,데이터과학] $\to$ 토픽1로 분류된 단어중에 `손흥민`은 하나도 없음 $\to$ 나는 토픽1이 아닌가? 

`-` `doc1-손흥민`이 토픽1에서 뽑혓다고 보는게 타당한지 토픽2에서 뽑혔다고 보는게 타당하지 따져보자.

- 토픽1의 타당성: (`doc1`에 토픽1이 포함되어 있는 비율) $\times$ (토픽1에서 `손흥민`이라는 단어를 뽑을 확률) = 1 $\times$ 0
- 토픽2의 타당성: (`doc1`에 토픽2가 포함되어 있는 비율) $\times$ (토픽2에서 `손흥민`이라는 단어를 뽑을 확률) = 0 $\times$ 0

> 둘 다 $0$ 이라서 비긴거야?? 그런데 토픽1이 그나마 타당한거아냐? 

`-` 수정된 타당성 

- 토픽1의 타당성: (문서1에 토픽1이 포함되어 있는 비율) $\times$ (토픽1에서 손흥민이라는 단어를 뽑을 확률) = 1 $\times$ 0.001
- 토픽2의 타당성: (문서1에 토픽2이 포함되어 있는 비율) $\times$ (토픽2에서 손흥민이라는 단어를 뽑을 확률) = 0.001 $\times$ 0.001

> $\theta_0$의 생각: 나는 토픽1인듯해

`-` 업데이트 

In [122]:
θ['doc1'][0]

2

In [123]:
θ['doc1'][0] = 1

#### stage1: ***${\boldsymbol \theta}_0[1]$의 고민: 나는 토픽1인가 토픽2인가?***

In [124]:
θ

{'doc1': [1, 1], 'doc2': [1, 2], 'doc3': [2, 1]}

In [125]:
D

{'doc1': ['손흥민', '골'], 'doc2': ['골', '확률'], 'doc3': ['확률', '데이터과학']}

`-` `doc1-골`의 (수정된) 타당성

- 토픽1의 타당성: (`doc1`에 토픽1이 포함되어 있는 비율) $\times$ (토픽1에서 `골`이라는 단어를 뽑을 확률) = 1 $\times$ 1/3
- 토픽2의 타당성: (`doc1`에 토픽2가 포함되어 있는 비율) $\times$ (토픽2에서 `골`이라는 단어를 뽑을 확률) = 0.001 $\times$ 0.001

#### ***...***

### ***단계3: 모든 $t \in \mathbb{N}_0$ 에 대하여 단계2를 반복***